#Load Drive and Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from google.colab import files

In [ ]:
import os
os.chdir('/content/drive/My Drive/TabDDPM')

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/TabDDPM')

#My dataset

In [ ]:
nameOfDataset="..."

!mkdir Results

%cd data
!mkdir {nameOfDataset}
%cd ..
%cd exp
!mkdir {nameOfDataset}
%cd ..

mkdir: cannot create directory ‘Results’: File exists
/content/drive/My Drive/TabDDPM/data
mkdir: cannot create directory ‘nba’: File exists
/content/drive/My Drive/TabDDPM
/content/drive/My Drive/TabDDPM/exp
mkdir: cannot create directory ‘nba’: File exists
/content/drive/My Drive/TabDDPM


In [ ]:
nameOfTest="..."

%cd Results
!mkdir {nameOfTest}
%cd {nameOfTest}
!mkdir Real
%cd ..
%cd ..

In [ ]:
filename = "..."   # use the name .csv file: nameOfDataset.csv

# Check if the file already exists
if not os.path.exists(filename):
    print(f"{filename} not found. Please upload it.")
    files.upload()

data = pd.read_csv(filename)
data = data.dropna().reset_index(drop=True)
attributes_to_ignore = ['...']
data = data.drop(attributes_to_ignore, axis=1)
cat_columns = ['...']
data

In [ ]:
def train_validate_test_split(df, train_percent=0.99, validate_percent=0.01, seed=1):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, train_end, validate, validate_end-train_end, test, m-validate_end

trainingSet, train_size, ValidationSet, val_size, testSet, test_size = train_validate_test_split(data,seed=5)

In [ ]:
def pandasDataFrameToNumpy(nameOfDataset, nameOfTest, trainingSet, ValidationSet, testSet):

    ##Training Set To Numpy in folder "Results\nameOfTest\Real\"
    labels= trainingSet.iloc[:,-1]
    label_c_name = labels.name
    trainingData = trainingSet.drop(trainingSet.columns[-1],axis = 1)
    categoricalData = trainingSet[cat_columns]
    cat_c_names = list(categoricalData.columns)
    numericalData = trainingData.drop(cat_columns,axis = 1)
    num_c_names = list(numericalData.columns)
    numericalData = numericalData.to_numpy(dtype='float32')


    d_dir = os.path.join("data", nameOfDataset)
    if not os.path.exists(d_dir):
        os.makedirs(d_dir)
    r_dir = os.path.join("Results", nameOfTest, "Real")
    if not os.path.exists(r_dir):
        os.makedirs(r_dir)

    np.save(os.path.join(d_dir, "X_num_train.npy"), numericalData)
    np.save(os.path.join(d_dir, "X_cat_train.npy"), categoricalData, allow_pickle=True)
    np.save(os.path.join(d_dir, "y_train.npy"), labels)

    np.save(os.path.join(r_dir, "X_num_train.npy"), numericalData)
    np.save(os.path.join(r_dir, "X_cat_train.npy"), categoricalData, allow_pickle=True)
    np.save(os.path.join(r_dir, "y_train.npy"), labels)

    ##Validation Set To Numpy in folder "Results\nameOfTest\Real\"
    labels= ValidationSet.iloc[:,-1]
    validationData = ValidationSet.drop(ValidationSet.columns[-1],axis = 1)
    categoricalData = validationData[cat_columns]
    numericalData = validationData.drop(cat_columns,axis = 1)
    numericalData = numericalData.to_numpy(dtype='float32')

    np.save(os.path.join(d_dir, "X_num_val.npy"), numericalData)
    np.save(os.path.join(d_dir, "X_cat_val.npy"), categoricalData, allow_pickle=True)
    np.save(os.path.join(d_dir, "y_val.npy"), labels)

    np.save(os.path.join(r_dir, "X_num_val.npy"), numericalData)
    np.save(os.path.join(r_dir, "X_cat_val.npy"), categoricalData, allow_pickle=True)
    np.save(os.path.join(r_dir, "y_val.npy"), labels)


    ##Test Set To Numpy in folder "Results\nameOfTest\Real\"
    labels= testSet.iloc[:,-1]
    testData = testSet.drop(testSet.columns[-1],axis = 1)
    categoricalData = testData[cat_columns]
    numericalData = testData.drop(cat_columns,axis = 1)
    numericalData = numericalData.to_numpy(dtype='float32')


    np.save(os.path.join(d_dir, "X_num_test.npy"), numericalData)
    np.save(os.path.join(d_dir, "X_cat_test.npy"), categoricalData, allow_pickle=True)
    np.save(os.path.join(d_dir, "y_test.npy"), labels)

    np.save(os.path.join(r_dir, "X_num_test.npy"), numericalData)
    np.save(os.path.join(r_dir, "X_cat_test.npy"), categoricalData, allow_pickle=True)
    np.save(os.path.join(r_dir, "y_test.npy"), labels)

    column_names = cat_c_names + num_c_names + [label_c_name]
    return column_names

In [ ]:
column_names = pandasDataFrameToNumpy(nameOfDataset, nameOfTest, trainingSet, ValidationSet, testSet)

In [ ]:
column_names

In [ ]:
#use your nameOfDataset and nameOfTest and adjust your parameters

toml_text = """
seed = 1
parent_dir = "exp/{nameOfDataset}/{nameOfTest}"
real_data_path = "data/{nameOfDataset}/"
model_type = "mlp"
num_numerical_features = ...
device = "cuda:0"

[model_params]
num_classes=0
is_y_cond = false

[model_params.rtdl_params]
d_layers = [
    512,
    1028,
    256,
]
dropout = 0.0

[diffusion_params]
num_timesteps = 1000
gaussian_loss_type = "mse"
scheduler = "cosine"

[train.main]
steps = 8000
lr = 0.001
weight_decay = 1e-05
batch_size = 100

[train.T]
seed = 0
normalization = "quantile"
num_nan_policy = "__none__"
cat_nan_policy = "__none__"
cat_min_frequency = "__none__"
cat_encoding = "__none__"
y_policy = "default"

[sample]
num_samples = 300
batch_size = 100
seed = 0

[eval.type]
eval_model = "catboost"
eval_type = "synthetic"

[eval.T]
seed = 0
normalization = "__none__"
num_nan_policy = "__none__"
cat_nan_policy = "__none__"
cat_min_frequency = "__none__"
cat_encoding = "__none__"
y_policy = "default"
"""

toml_dir = os.path.join("exp", nameOfDataset, nameOfTest)
os.makedirs(toml_dir, exist_ok=True)
with open(os.path.join(toml_dir, "config.toml"), "w") as f:
    f.write(toml_text)

In [ ]:
print(train_size)
print(val_size)
print(test_size)

In [ ]:
info_txt="""
{
    "name": "...",
    "id": "...",
    "task_type": "regression",
    "n_num_features": ...,
    "n_cat_features": ...,
    "train_size": ...,
    "val_size": ...,
    "test_size": ...
}
"""

info_dir = os.path.join("data", nameOfDataset)
with open(os.path.join(info_dir, "info.json"), "w") as f:
    f.write(info_txt)

In [ ]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
!pip install catboost category-encoders dython icecream libzero optuna pyarrow rtdl scipy skorch tomli-w tomli tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 115.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!python -m scripts.pipeline --config exp/nba/nbaTest/config.toml --train --sample

In [ ]:
num_file_path =  '/content/drive/MyDrive/TabDDPM/exp/nba/nbaTest/X_num_train.npy'
X_num_gen = np.load(num_file_path, allow_pickle=True)

In [ ]:
cat_file_path =  '/content/drive/MyDrive/TabDDPM/exp/nba/nbaTest/X_cat_train.csv'
df_loaded = pd.read_csv(cat_file_path)
X_cat_gen = df_loaded.values

In [ ]:
y_file_path = '/content/drive/MyDrive/TabDDPM/exp/nba/nbaTest/y_train.npy'
y_gen = np.load(y_file_path, allow_pickle=True)

In [ ]:
if y_gen.shape[1:] == ():
    y_gen = y_gen.reshape(-1, 1)
if X_cat_gen.shape[1:] == ():
    X_cat_gen = X_cat_gen.reshape(-1, 1)
if X_num_gen.shape[1:] == ():
    X_num_gen = X_num_gen.reshape(-1, 1)

In [ ]:
final_df = pd.DataFrame(np.concatenate([X_cat_gen, X_num_gen, y_gen], axis=1), columns=column_names)
final_df